# ESC 2020 SUMMER

## 데이터 분석 1차 발표 - Preprocessing & EDA
## 7조 
![sh](ChilliShrimp.png)

# Overview

[github link](https://github.com/njj06135/Baseball_ChilliShrimp)

- 조원 소개
- 분석 방식
- 향후 계획

![kbo](open_graph_kbo.png)

# 1. 조원 소개

- 조장 : ★곽현지★ ESC 22기



- 조원 : 오태환(22기) 김채형(23기) 박상재(24기) 이승준(24기)

# 2. 분석 방식

## 두 가지 제안!

1. 2020년 7월 19일 까지가 기준이니, 모든 년도의 팀 기록을 7월 19일 이전과 이후로 나누어서 7월 19일 이전은 X변수로 7월 19일 이후는 승률만 계산해 y로 넣고 2016~2019로 트레인 시킨 후 2020년 기록을 넣고 prediction을 하자



2. 팀 성적은 결국 선수 성적의 합이다. 선수들의 잔여 경기 성적을 예측한 뒤, 그것을 합쳐서 팀 성적을 만든 후 승률을 계산하자. 타자와 투수의 성적을 합해 성적을 계산하기 때문에 자동으로 타율, 방어율은 계산이 된다.


1번 방식의 문제점은 row가 겨우 30개 밖에 안된다는 것이다! 이렇게 적은 데이터로는 좋은 결과를 낼 수 없다고 생각해서 2번이 좀 더 적절한 방식이라고 생각했다.

그렇다면 우선 x는 뭐고 y는 뭘로 해야할까?
투수의 경우는 명확한 y값이 있다. 바로 실점이다. 자책점이 아닌 실점을 사용한 이유는 연구를 통해 자책점보다 실점이 팀 승률과의 상관계수가 더 높다는 것으로 알려져있기 때문이다 - 그래프 찾아서 넣을 예정 -

그렇다면 타자의 y는 무엇으로 해야할까? 득점이나 타점은 선수 본인만의 능력이 아닌 팀 내 다른 선수들의 영향이 포함되어있다. 득점은 다른 선수가 쳐줘야 들어올 수 있고 타점은 다른 선수가 루상에 있어줘야 올릴 수 있기 때문이다. 그래서 그 선수가 얼마나 팀의 득점에 기여했는가를 어떻게 알 수 있을까를 찾아보던 중, XR이란 스탯을 발견하게 되었다.

# XR(추정 득점)?

(1루타×0.5)+(2루타×0.72)+(3루타×1.04)+(홈런×1.44)+{(사구+볼넷－고의사구)×0.34}+(고의

   사구×0.25)+(도루×0.18)－(도루실패×0.32)－{(타수－안타－삼진)×0.09}－(삼진×0.098)－(병살타×0.37)+(희생

   플라이×0.37)+(희생번트×0.04)



출처: https://jagaevi.tistory.com/69 [기억상실증] 

이러한 식으로 계산되는 XR은 그 선수가 팀의 득점에 얼마나 기여했는지를 회귀분석을 통해 계산한 값이다.
https://www.baseballthinkfactory.org/btf/scholars/furtado/articles/IntroducingXR.htm

링크에서 볼 수 있듯, 팀 내 모든 선수의 XR을 더하면 팀의 득점과 거의 비슷한 값을 얻을 수 있다. 따라서 타자의 y로는 XR을 계산해 사용하도록 하겠다.

# X값들은 비율스탯으로 바꾸기

팀마다 경기수가 다르고 선수마다 들어간 타석수, 투입된 이닝 수가 다르다. 그렇기 때문에 절대적인 양으로 X값을 넣는다면 그러한 차이 때문에 왜곡된 값이 나올 수 있다고 생각했다. 그래서 타율, 출루율, 장타율, OPS, 9이닝당 볼넷, 9이닝당 삼진, 9이닝당 홈런 등 비율스탯으로 바꾸어 넣기로 했다.

Cf) OPS란?

# 그렇다면 결국 y값을 구하기 위해선 2020년 7월 19일 이후의 X값이 필요하잖아?

BABIP이론

타자들은 운이 좋아서 안타를 칠 수도, 투수들은 운이 좋아서 아웃을 잡을 수도 있다. 하지만 그러한 운은 결국 평균으로 회귀한다. 그렇다면 7월 19일까지 운이 좋았던 선수들은 잔여 성적을 과소평가, 운이 좋지 않았던 선수들은 잔여 성적을 과대평가 해야하지 않을까? 그래서 사용하기로 한 지표가 BABIP이다. BABIP 이론에 따르면, 결국 그 선수의 BABIP는 본인의 통산 BABIP를 따라가게 되어있다. 그렇기 때문에 7월 19일까지의 BABIP를 계산해 잔여경기를 마치면 BABIP가 그 선수의 통산 평균이 되도록 맞춰주려고 한다. 우선 PA를 시계열 분석을 통해 예측한다. 그리고 그 선수의 통산 인플레이 타구 비율을 곱해 인플레이 타구의 수를 구하고, 안타의 갯수를 통산 BABIP가 되도록 맞춰준다. 그리고 볼넷과 삼진, 장타 등은 그에 맞추어 통산 비율을 곱해 구하기로 한다. 투수도 마찬가지이다. 시계열 분석을 통해 등판 이닝을 구하고, BABIP에 맞게 피안타를 구한 다음, 볼넷과 삼진 장타 등은 통산 비율을 곱해 구한다

# 그렇다면 팀 승률은 어떻게?

* [피타고리안 승률](https://sports.v.daum.net/v/20170802145321697)

위의 과정을 통해 우리는 타자들의 XR들의 합으로 나타내어진 그 팀의 총 득점, 투수들의 실점의 합으로 나타내어진 그 팀의 총 실점을 예측했다. 그리고 팀의 득점과 실점을 통해 팀의 승률을 구하는 식이 바로 피타고리안 승률이다.

운이 좋은 팀도 있을 수 있고 운이 나쁜 팀이 있을 수도 있다. 하지만 결국 스코어로 사용되는 것은 MSE이기 때문에 그 중앙이라고도 할 수 있는 피타고리안 승률을 사용하는 것이 옳은 방법이라고 봤다.

# 3. 향후 계획

- 추가 EDA : 시각화, Scaling, 파생변수 만들기
- 모델링
- 최종 데이터 만들기
- 혹시 입상을 하면 발표 ppt 만들기